# Import thư viện và thiết lập đường dẫn

In [ ]:
import os
import yaml
import glob
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import random
import cv2
from ultralytics import YOLO
import torch

base_dir = "D:/Bi_data/21031271/ThiGiacMayTinh/Project_Cuoi_Ky/FB5/Data"
train_dir = os.path.join(base_dir, "processed_images")
val_img_dir = os.path.join(base_dir, "val_images")
test_img_dir = os.path.join(base_dir, "test_images")
data_yaml = os.path.join(base_dir, "data.yaml")
model_dir = os.path.join(base_dir, "MODEL")
# Tạo thư mục lưu mô hình nếu chưa có
os.makedirs(model_dir, exist_ok=True)
print("Đã thiết lập đường dẫn và thư mục mô hình.")

Đã thiết lập đường dẫn và thư mục mô hình.


In [ ]:
# Kiểm tra thư mục dữ liệu
for dir_path, name in [
    (train_dir, "Train images"),
    (val_img_dir, "Validation images"),
    (test_img_dir, "Test images")
]:
    if not os.path.exists(dir_path):
        raise FileNotFoundError(f"{name} không tồn tại: {dir_path}")
    if not os.listdir(dir_path):
        raise FileNotFoundError(f"{name} rỗng: {dir_path}")
print("Tất cả thư mục dữ liệu tồn tại và không rỗng.")

Tất cả thư mục dữ liệu tồn tại và không rỗng.


In [ ]:
# Kiểm tra file data.yaml
if not os.path.exists(data_yaml):
    raise FileNotFoundError(f"File data.yaml không tồn tại: {data_yaml}")

with open(data_yaml, 'r') as f:
    data_config = yaml.safe_load(f)

expected_config = {
    'path': base_dir,
    'train': 'processed_images',
    'val': 'val_images',
    'test': 'test_images',
    'nc': 2,
    'names': ['face', 'mask']
}
for key, value in expected_config.items():
    if data_config.get(key) != value:
        print(f"Cảnh báo: data.yaml có cấu hình không đúng cho {key}. Thực tế: {data_config.get(key)}, Kỳ vọng: {value}")
    else:
        print(f"{key}: Hợp lệ ({value})")

path: Hợp lệ (D:/Bi_data/21031271/ThiGiacMayTinh/Project_Cuoi_Ky/FB5/Data)
train: Hợp lệ (processed_images)
val: Hợp lệ (val_images)
test: Hợp lệ (test_images)
nc: Hợp lệ (2)
names: Hợp lệ (['face', 'mask'])


In [ ]:
# Kiểm tra số lượng file và tính khớp nhau
train_images = [f for f in os.listdir(train_dir) if f.endswith('.png')]
train_labels = [f for f in os.listdir(train_dir) if f.endswith('.txt')]
val_images = [f for f in os.listdir(val_img_dir) if f.endswith('.png')]
val_labels = [f for f in os.listdir(val_img_dir) if f.endswith('.txt')]
test_images = [f for f in os.listdir(test_img_dir) if f.endswith('.png')]
test_labels = [f for f in os.listdir(test_img_dir) if f.endswith('.txt')]

print("Kiểm tra dữ liệu:")
print(f"- Train images: {len(train_images)}")
print(f"- Train labels: {len(train_labels)}")
print(f"- Validation images: {len(val_images)}")
print(f"- Validation labels: {len(val_labels)}")
print(f"- Test images: {len(test_images)}")
print(f"- Test labels: {len(test_labels)}")

# Kiểm tra cân bằng dữ liệu (đếm ảnh có/không khẩu trang)
def count_mask_images(img_dir, labels):
    mask_count = 0
    no_mask_count = 0
    for label in labels:
        with open(os.path.join(img_dir, label), 'r') as f:
            lines = f.readlines()
            has_mask = any(line.startswith('1') for line in lines)
            if has_mask:
                mask_count += 1
            else:
                no_mask_count += 1
    return mask_count, no_mask_count

train_mask, train_no_mask = count_mask_images(train_dir, train_labels)
val_mask, val_no_mask = count_mask_images(val_img_dir, val_labels)
test_mask, test_no_mask = count_mask_images(test_img_dir, test_labels)

print("\nCân bằng dữ liệu:")
print(f"- Train: {train_mask} ảnh có khẩu trang, {train_no_mask} ảnh không khẩu trang")
print(f"- Validation: {val_mask} ảnh có khẩu trang, {val_no_mask} ảnh không khẩu trang")
print(f"- Test: {test_mask} ảnh có khẩu trang, {test_no_mask} ảnh không khẩu trang")

# Kiểm tra file không khớp
def check_matching(img_dir, name):
    images = [f for f in os.listdir(img_dir) if f.endswith('.png')]
    labels = [f for f in os.listdir(img_dir) if f.endswith('.txt')]
    label_names = [f.replace('.txt', '') for f in labels]
    
    missing_labels = [img for img in images if img.replace('.png', '') not in label_names]
    missing_images = [label for label in labels if not os.path.exists(os.path.join(img_dir, label.replace('.txt', '.png')))]
    
    return missing_labels, missing_images

train_missing_labels, train_missing_images = check_matching(train_dir, "Train")
val_missing_labels, val_missing_images = check_matching(val_img_dir, "Validation")
test_missing_labels, test_missing_images = check_matching(test_img_dir, "Test")

if any([train_missing_labels, train_missing_images, val_missing_labels, val_missing_images, test_missing_labels, test_missing_images]):
    print("\nFile không khớp:")
    for split, missing_labels, missing_images in [
        ("Train", train_missing_labels, train_missing_images),
        ("Validation", val_missing_labels, val_missing_images),
        ("Test", test_missing_labels, test_missing_images)
    ]:
        if missing_labels:
            print(f"{split} - Ảnh không có nhãn:")
            for img in missing_labels:
                print(f"- {img}")
        if missing_images:
            print(f"{split} - Nhãn không có ảnh:")
            for label in missing_images:
                print(f"- {label}")
    
    delete_files = input("\nBạn có muốn xóa các file không khớp? (y/n): ").lower() == 'y'
    if delete_files:
        for img in train_missing_labels:
            os.remove(os.path.join(train_dir, img))
            print(f"Đã xóa: {img}")
        for label in train_missing_images:
            os.remove(os.path.join(train_dir, label))
            print(f"Đã xóa: {label}")
        for img in val_missing_labels:
            os.remove(os.path.join(val_img_dir, img))
            print(f"Đã xóa: {img}")
        for label in val_missing_images:
            os.remove(os.path.join(val_img_dir, label))
            print(f"Đã xóa: {label}")
        for img in test_missing_labels:
            os.remove(os.path.join(test_img_dir, img))
            print(f"Đã xóa: {img}")
        for label in test_missing_images:
            os.remove(os.path.join(test_img_dir, label))
            print(f"Đã xóa: {label}")
        
        # Cập nhật danh sách
        train_images = [f for f in os.listdir(train_dir) if f.endswith('.png')]
        train_labels = [f for f in os.listdir(train_dir) if f.endswith('.txt')]
        val_images = [f for f in os.listdir(val_img_dir) if f.endswith('.png')]
        val_labels = [f for f in os.listdir(val_img_dir) if f.endswith('.txt')]
        test_images = [f for f in os.listdir(test_img_dir) if f.endswith('.png')]
        test_labels = [f for f in os.listdir(test_img_dir) if f.endswith('.txt')]
        print("\nDanh sách sau khi xóa:")
        print(f"- Train images: {len(train_images)}")
        print(f"- Train labels: {len(train_labels)}")
        print(f"- Validation images: {len(val_images)}")
        print(f"- Validation labels: {len(val_labels)}")
        print(f"- Test images: {len(test_images)}")
        print(f"- Test labels: {len(test_labels)}")
else:
    print("\nTất cả file khớp hoàn toàn.")

Kiểm tra dữ liệu:
- Train images: 5984
- Train labels: 5984
- Validation images: 1282
- Validation labels: 1282
- Test images: 1283
- Test labels: 1283

Cân bằng dữ liệu:
- Train: 2856 ảnh có khẩu trang, 3128 ảnh không khẩu trang
- Validation: 597 ảnh có khẩu trang, 685 ảnh không khẩu trang
- Test: 643 ảnh có khẩu trang, 640 ảnh không khẩu trang

Tất cả file khớp hoàn toàn.


In [ ]:
# Kiểm tra tính hợp lệ của nhãn
def check_label_content(img_dir, name, num_samples=5):
    label_files = [f for f in os.listdir(img_dir) if f.endswith('.txt')]
    print(f"\nKiểm tra nội dung {num_samples} file nhãn trong {name}:")
    for label_file in label_files[:num_samples]:
        with open(os.path.join(img_dir, label_file), 'r') as f:
            content = f.read().strip()
            if not content:
                print(f"- {label_file}: File rỗng!")
                continue
            lines = content.split('\n')
            valid = True
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"- {label_file}: Dòng không đúng định dạng: {line}")
                    valid = False
                    continue
                try:
                    class_id, x, y, w, h = map(float, parts)
                    if not (0 <= class_id <= 1) or not (0 <= x <= 1) or not (0 <= y <= 1) or not (0 <= w <= 1) or not (0 <= h <= 1):
                        print(f"- {label_file}: Giá trị ngoài khoảng [0, 1]: {line}")
                        valid = False
                except ValueError:
                    print(f"- {label_file}: Giá trị không hợp lệ: {line}")
                    valid = False
            if valid:
                print(f"- {label_file}: Hợp lệ")

check_label_content(train_dir, "Train")
check_label_content(val_img_dir, "Validation")
check_label_content(test_img_dir, "Test")


Kiểm tra nội dung 5 file nhãn trong Train:
- mask_1000_empty.txt: Hợp lệ
- mask_1001_cloth.txt: Hợp lệ
- mask_1004_inpaint.txt: Hợp lệ
- mask_1007_surgical.txt: Hợp lệ
- mask_1008_KN95.txt: Hợp lệ

Kiểm tra nội dung 5 file nhãn trong Validation:
- mask_1006_cloth.txt: Hợp lệ
- mask_1009_surgical_green.txt: Hợp lệ
- mask_1021_gas.txt: Hợp lệ
- mask_1023_gas.txt: Hợp lệ
- mask_1025_empty.txt: Hợp lệ

Kiểm tra nội dung 5 file nhãn trong Test:
- mask_1005_KN95.txt: Hợp lệ
- mask_1010_empty.txt: Hợp lệ
- mask_1050_surgical.txt: Hợp lệ
- mask_1053_gas.txt: Hợp lệ
- mask_1056_cloth.txt: Hợp lệ


In [ ]:
# Xóa cache dataset
cache_files = glob.glob(os.path.join(base_dir, "*.cache"))
for cache_file in cache_files:
    try:
        os.remove(cache_file)
        print(f"Đã xóa cache: {cache_file}")
    except Exception as e:
        print(f"Lỗi khi xóa cache {cache_file}: {e}")

# Kiểm tra GPU và khởi tạo mô hình

In [ ]:


if torch.cuda.is_available():
    device = 0  # Sử dụng GPU RTX 2050
    print(f"GPU khả dụng: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("Không tìm thấy GPU, sử dụng CPU.")

# Khởi tạo mô hình
model = YOLO("yolov8n.pt")  # Sử dụng YOLOv8 nano pretrained

# Thông số huấn luyện
train_params = {
    "data": data_yaml,
    "epochs": 100,  # Tăng epochs
    "imgsz": 256,
    "batch": 32,  # Tăng batch size
    "device": device,
    "patience": 20,  # Early stopping
    "name": "yolo_face_mask",
    "optimizer": "AdamW",  # Tối ưu hơn SGD
    "lr0": 0.01,  # Learning rate ban đầu
    "lrf": 0.1,  # Learning rate cuối
    "hsv_h": 0.015,  # Tăng cường dữ liệu
    "hsv_s": 0.7,
    "hsv_v": 0.4,
    "degrees": 10.0,
    "translate": 0.1,
    "scale": 0.5,
    "shear": 2.0,
    "flipud": 0.5,
    "fliplr": 0.5,
    "mosaic": 1.0,
    "mixup": 0.0
}

# Huấn luyện và đo thời gian
start_time = time.time()
results = model.train(**train_params)
end_time = time.time()
training_time = end_time - start_time
avg_epoch_time = training_time / train_params["epochs"]

# Hiển thị thông số huấn luyện
print("\n=== Thông số huấn luyện ===")
print(f"Số epoch: {train_params['epochs']}")
print(f"Batch size: {train_params['batch']}")
print(f"Kích thước ảnh: {train_params['imgsz']}x{train_params['imgsz']}")
print(f"Thiết bị: {device if device == 'cpu' else torch.cuda.get_device_name(device)}")
print(f"Tổng thời gian huấn luyện: {training_time:.2f} giây")
print(f"Thời gian trung bình mỗi epoch: {avg_epoch_time:.2f} giây")

Không tìm thấy GPU, sử dụng CPU.


100%|██████████| 6.25M/6.25M [00:02<00:00, 2.59MB/s]


New https://pypi.org/project/ultralytics/8.3.144 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.128  Python-3.12.3 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-12500H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/Bi_data/21031271/ThiGiacMayTinh/Project_Cuoi_Ky/FB5/Data\data.yaml, degrees=10.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yol

train: Scanning D:\Bi_data\21031271\ThiGiacMayTinh\Project_Cuoi_Ky\FB5\Data\processed_images... 5984 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5984/5984 [00:12<00:00, 494.42it/s]


train: New cache created: D:\Bi_data\21031271\ThiGiacMayTinh\Project_Cuoi_Ky\FB5\Data\processed_images.cache
val: Fast image access  (ping: 0.00.0 ms, read: 3.60.7 MB/s, size: 47.9 KB)


...

    100/100         0G     0.3158     0.1583     0.9505         46        256: 100%|██████████| 187/187 [03:37<00:00,  1.16s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:19<00:00,  1.06it/s]

                   all       1282       1879      0.999      0.998      0.995      0.944



100 epochs completed in 6.665 hours.
Optimizer stripped from runs\detect\yolo_face_mask\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\yolo_face_mask\weights\best.pt, 6.2MB

Validating runs\detect\yolo_face_mask\weights\best.pt...
Ultralytics 8.3.128  Python-3.12.3 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-12500H)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:17<00:00,  1.20it/s]


                   all       1282       1879          1      0.998      0.995      0.946
                  face       1282       1282          1      0.998      0.995      0.935
                  mask        597        597          1      0.998      0.995      0.956
Speed: 0.2ms preprocess, 10.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\yolo_face_mask

=== Thông số huấn luyện ===
Số epoch: 100
Batch size: 32
Kích thước ảnh: 256x256
Thiết bị: cpu
Tổng thời gian huấn luyện: 24036.50 giây
Thời gian trung bình mỗi epoch: 240.37 giây


# Hiển thị loss và đánh giá mô hình

In [ ]:

print("\nLoss cuối cùng:")
run_dir = results.save_dir
csv_path = os.path.join(run_dir, "results.csv")
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    last_epoch = df.iloc[-1]
    print(f"Box loss: {last_epoch.get('train/box_loss', 'N/A'):.4f}")
    print(f"Cls loss: {last_epoch.get('train/cls_loss', 'N/A'):.4f}")
    print(f"Dfl loss: {last_epoch.get('train/dfl_loss', 'N/A'):.4f}")
else:
    print("Không tìm thấy results.csv.")

# Đánh giá mô hình trên tập validation
print("\n=== Đánh giá trên tập validation ===")
val_results = model.val(data=data_yaml, imgsz=train_params["imgsz"], batch=train_params["batch"], device=train_params["device"])

# Đánh giá mô hình trên tập test
print("\n=== Đánh giá trên tập test ===")
test_results = model.val(data=data_yaml, imgsz=train_params["imgsz"], batch=train_params["batch"], device=train_params["device"], split="test")

# Hiển thị chỉ số đánh giá
print("\nChỉ số đánh giá (Validation):")
print(f"mAP@0.5: {val_results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {val_results.box.map:.4f}")
for i, (p, r, f1) in enumerate(zip(val_results.box.p, val_results.box.r, val_results.box.f1)):
    print(f"Lớp {val_results.names[i]}:")
    print(f"  Precision: {p:.4f}")
    print(f"  Recall: {r:.4f}")
    print(f"  F1-score: {f1:.4f}")

print("\nChỉ số đánh giá (Test):")
print(f"mAP@0.5: {test_results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {test_results.box.map:.4f}")
for i, (p, r, f1) in enumerate(zip(test_results.box.p, test_results.box.r, test_results.box.f1)):
    print(f"Lớp {test_results.names[i]}:")
    print(f"  Precision: {p:.4f}")
    print(f"  Recall: {r:.4f}")
    print(f"  F1-score: {f1:.4f}")



Loss cuối cùng:
Box loss: 0.3158
Cls loss: 0.1583
Dfl loss: 0.9505

=== Đánh giá trên tập validation ===
Ultralytics 8.3.128  Python-3.12.3 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-12500H)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 590.0285.3 MB/s, size: 49.7 KB)


val: Scanning D:\Bi_data\21031271\ThiGiacMayTinh\Project_Cuoi_Ky\FB5\Data\val_images.cache... 1282 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1282/1282 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 41/41 [00:18<00:00,  2.26it/s]


                   all       1282       1879          1      0.998      0.995      0.946
                  face       1282       1282          1      0.998      0.995      0.935
                  mask        597        597          1      0.998      0.995      0.956
Speed: 0.2ms preprocess, 10.9ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\yolo_face_mask2

=== Đánh giá trên tập test ===
Ultralytics 8.3.128  Python-3.12.3 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-12500H)
val: Fast image access  (ping: 0.00.0 ms, read: 4.11.4 MB/s, size: 50.3 KB)


val: Scanning D:\Bi_data\21031271\ThiGiacMayTinh\Project_Cuoi_Ky\FB5\Data\test_images... 1283 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1283/1283 [00:02<00:00, 474.22it/s]

val: New cache created: D:\Bi_data\21031271\ThiGiacMayTinh\Project_Cuoi_Ky\FB5\Data\test_images.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 41/41 [00:17<00:00,  2.33it/s]


                   all       1283       1926      0.999      0.999      0.995      0.946
                  face       1283       1283      0.999      0.999      0.995      0.933
                  mask        643        643      0.999          1      0.995      0.959
Speed: 0.2ms preprocess, 10.5ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\yolo_face_mask3

Chỉ số đánh giá (Validation):
mAP@0.5: 0.9950
mAP@0.5:0.95: 0.9456
Lớp face:
  Precision: 1.0000
  Recall: 0.9978
  F1-score: 0.9989
Lớp mask:
  Precision: 0.9995
  Recall: 0.9983
  F1-score: 0.9989

Chỉ số đánh giá (Test):
mAP@0.5: 0.9950
mAP@0.5:0.95: 0.9464
Lớp face:
  Precision: 0.9992
  Recall: 0.9986
  F1-score: 0.9989
Lớp mask:
  Precision: 0.9991
  Recall: 1.0000
  F1-score: 0.9995


# Hiển thị biểu đồ đánh giá

In [ ]:

print("\n=== Hiển thị biểu đồ ===")
plot_files = [
    "confusion_matrix.png",
    "PR_curve.png",
    "F1_curve.png",
    "results.png"
]

for plot_file in plot_files:
    plot_path = os.path.join(run_dir, plot_file)
    if os.path.exists(plot_path):
        img = np.array(Image.open(plot_path))
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.axis('off')
        plt.title(plot_file)
        plt.show()
    else:
        print(f"Không tìm thấy: {plot_path}")


=== Hiển thị biểu đồ ===


<Figure size 800x600 with 1 Axes>

<Figure size 800x600 with 1 Axes>

<Figure size 800x600 with 1 Axes>

<Figure size 800x600 with 1 Axes>

# Dự đoán trên ảnh mẫu từ tập test

In [ ]:

print("\n=== Dự đoán trên 5 ảnh mẫu từ tập test ===")
test_images = [f for f in os.listdir(test_img_dir) if f.endswith('.png')]
sample_images = random.sample(test_images, min(5, len(test_images)))

for img_name in sample_images:
    img_path = os.path.join(test_img_dir, img_name)
    results = model.predict(img_path, imgsz=256, conf=0.5, device=device)
    
    # Hiển thị ảnh với bounding box
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()
        confidences = result.boxes.conf.cpu().numpy()
        
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = map(int, box)
            label = f"{result.names[int(cls)]} {conf:.2f}"
            color = (0, 255, 0) if cls == 0 else (0, 0, 255)  # Xanh cho face, đỏ cho mask
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(f"Dự đoán: {img_name}")
    plt.axis('off')
    plt.show()


=== Dự đoán trên 5 ảnh mẫu từ tập test ===

image 1/1 D:\Bi_data\21031271\ThiGiacMayTinh\Project_Cuoi_Ky\FB5\Data\test_images\mask_4037_surgical_green.png: 256x256 1 face, 1 mask, 59.0ms
Speed: 1.8ms preprocess, 59.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)


<Figure size 600x600 with 1 Axes>


image 1/1 D:\Bi_data\21031271\ThiGiacMayTinh\Project_Cuoi_Ky\FB5\Data\test_images\no_mask_without_mask_1547.png: 256x256 1 face, 23.6ms
Speed: 0.6ms preprocess, 23.6ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 256)


<Figure size 600x600 with 1 Axes>


image 1/1 D:\Bi_data\21031271\ThiGiacMayTinh\Project_Cuoi_Ky\FB5\Data\test_images\no_mask_2342.png: 256x256 1 face, 27.0ms
Speed: 0.5ms preprocess, 27.0ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 256)


<Figure size 600x600 with 1 Axes>


image 1/1 D:\Bi_data\21031271\ThiGiacMayTinh\Project_Cuoi_Ky\FB5\Data\test_images\no_mask_1421.png: 256x256 1 face, 26.7ms
Speed: 0.5ms preprocess, 26.7ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 256)


<Figure size 600x600 with 1 Axes>


image 1/1 D:\Bi_data\21031271\ThiGiacMayTinh\Project_Cuoi_Ky\FB5\Data\test_images\mask_without_mask_1929_N95.png: 256x256 1 face, 1 mask, 28.2ms
Speed: 0.6ms preprocess, 28.2ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 256)


<Figure size 600x600 with 1 Axes>

In [11]:
# Cell 11: Lưu mô hình
model_path = os.path.join(model_dir, "yolo_face_mask.pt")
model.save(model_path)
print(f"\nHuấn luyện, đánh giá và dự đoán hoàn tất! Mô hình được lưu tại: {model_path}")


Huấn luyện, đánh giá và dự đoán hoàn tất! Mô hình được lưu tại: D:/Bi_data/21031271/ThiGiacMayTinh/Project_Cuoi_Ky/FB5/Data\MODEL\yolo_face_mask.pt
